In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import pandas as pd
import os

In [ ]:
# Set environment variables for reproducibility
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
torch.use_deterministic_algorithms(True, warn_only=True)

os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

torch.manual_seed(40)
torch.cuda.manual_seed_all(40)

In [ ]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'
device

In [ ]:
def train_model(model, train_loader, test_loader, num_epochs, loss_function, optimizer, batch_size, classes):
    for epoch in range(num_epochs):
        for i, (image, label) in enumerate(train_loader):
            image = image.to(device)
            label = label.to(device)
        
            optimizer.zero_grad()
            output = model(image)
            loss = loss_function(output, label)
            loss.backward()
            optimizer.step()
        
            if (i+1)%2500==0:
                print(f'Epoch {epoch+1}, Step {i+1}, Loss= {loss.item():.4f}')
    print('Training Complete')

    with torch.no_grad():
        n_correct = 0
        n_samples = 0
        n_class_correct = [0 for _ in range(10)]
        n_class_samples = [0 for _ in range(10)]
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            output = model(images)
            _, predicted = torch.max(output, 1)
            n_samples += labels.size(0)
            n_correct += (predicted==labels).sum().item()
        
            for i in range(batch_size):
                label = labels[i]
                pred = predicted[i]
                if (label==pred):
                    n_class_correct[label] += 1
                n_class_samples[label] += 1
    
    print(f'Accuracy of the network: {100 * n_correct/n_samples} %')

    for i in range(10):
        acc = 100.0*n_class_correct[i]/n_class_samples[i]
        print(f'Accuracy of {classes[i]} : {acc:.3f}%')

In [ ]:
epochs = 4
batch_size = 4
lr = 1e-3

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.1307], std=[0.3081])
])

train_dataset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

classes = ('t-shirt', 'trouser', 'pullover', 'dress', 'coat', 'sandal', 'shirt', 'sneaker', 'bag', 'boot')


In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16,32,3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(32*7*7, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )
    def forward(self, x):
        return self.model(x)

In [ ]:
model = CNN().to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr)

In [ ]:
train_model(model, train_loader, test_loader, epochs, loss_function, optimizer, batch_size, classes)